In [ ]:
# Importing of the necessary libraries

import csv
import cv2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, ELU
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D

from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#### First try with a basic fully connected one layer structure just to see data load and basic epoch run working

model = Sequential()
#model.add(Lambda(lambda x: (X_train / 255.0) - 0.5), input_shape=(160,320,3))
model.add(Flatten(input_shape=(160,320,3)))
#model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
#model.compile(loss='binary_crossentropy', optimizer='adam')
#model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(norm(X_train), y_train, validation_split=0.2, shuffle=True, nb_epoch=7)

#model.save('model.h5')


In [ ]:
# Adding right and left camera images to the dataset
# Also added flipped version of whole images
# As the third step only some part of the track added to whole data set. Then the data shuffled
# 4th step - adding only part of a track data got things worse. Recorded full track data added.
   
def data_add(path, file_path):
    augmented_images, augmented_measurements = [], []
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        first_line = f.readline()    # get rid of first row including string labels
        for row in reader:
            steering_center = float(row[3])

            # create adjusted steering measurements for the side camera images
            correction = 0.2 # this is a parameter to tune
            steering_left = steering_center + correction
            steering_right = steering_center - correction

            # read in images from center, left and right cameras

            img_center = cv2.imread(path + row[0].split('/')[-1])
            ## process_image(np.asarray(Image.open(path + row[0])))
            img_left = cv2.imread(path + row[1].split('/')[-1])
            ## process_image(np.asarray(Image.open(path + row[1])))
            img_right = cv2.imread(path + row[2].split('/')[-1])
            ## process_image(np.asarray(Image.open(path + row[2])))

            # add images and angles to data set
            augmented_images.append(img_center)
            augmented_images.append(img_left)
            augmented_images.append(img_right)

            augmented_measurements.append(steering_center)
            augmented_measurements.append(steering_left)
            augmented_measurements.append(steering_right)

            augmented_images.append(cv2.flip(img_center,1))
            augmented_images.append(cv2.flip(img_left,1))
            augmented_images.append(cv2.flip(img_right,1))

            augmented_measurements.append(-steering_center)
            augmented_measurements.append(-steering_left)
            augmented_measurements.append(-steering_right)
    return augmented_measurements, augmented_images

aug_measurements, aug_images = [],[]

aug_measurements, aug_images = data_add("data/IMG/", 'data/driving_log.csv')

# Adding personally created data - the necessity of this data explained in markup
# personal_measurements, personal_images = data_add("data_celal/IMG/", 'data_celal/driving_log.csv')
personal_measurements, personal_images = data_add("data_full_path/IMG/", 'data_full_path/driving_log.csv')

aug_measurements.extend(personal_measurements)
aug_images.extend(personal_images)

X_train = np.array(aug_images)  
y_train = np.array(aug_measurements)

In [ ]:
# Unit test : to see number of pictures, to see numbers match
print(len(X_train), len(y_train))


In [ ]:
# We have to shuffle before training because i have added specific parts of the track to the data set. 
# This data added by me should be shuffled so validation data will be balanced. 

from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

In [ ]:
# Try a modified VGGNet structure i found on the net.

# Cropping layer added as it is not necessary to include top and bottom portions of the pictures
# Also, network is too slow, it takes 8 hours to finish 20 epochs with using only test data given by Udacity

# Also a lambda layer added for normalization. W/o this layer the values feed into simulator with drive.py became
# meaningless. This is a must addition for this project.

model = Sequential()

model.add(Cropping2D(cropping=((70, 25), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

model.add(Lambda(lambda x: (x / 255.0) - 0.5))

model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(Convolution2D(32, 3, 3, activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
  
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
 
model.fit(X_train, y_train, batch_size=32, validation_split=0.2, shuffle=True, nb_epoch=15)

model.save('model-VGG_15_augmented-leftrightcam_personal_batch-32_split20.h5')


In [ ]:
# Modified existing structure to add more depth to convolution layer, starting from 16 to 128, 
# to extract more features out of images

# I also played with batch and validation split hyperparameters.

# It didn't effect the results much after first try.

model = Sequential()

model.add(Cropping2D(cropping=((70, 25), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

model.add(Lambda(lambda x: (x / 255.0) - 0.5))
model.add(Convolution2D(16, 3, 3, activation='relu'))
model.add(Convolution2D(32, 3, 3, activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(128, 3, 3, activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train, batch_size=128, validation_split=0.25, shuffle=True, nb_epoch=15)

model.save('model-mod_VGG_15_augmented-leftrightcam_personal_batch-128_split25.h5')


In [ ]:
# Comm.ai structure tried couple times. Loss values not meaningful after couple epochs. 
# I have to work on this structure and make more trial and errors.

model = Sequential()

model.add(Cropping2D(cropping=((70, 25), (0, 0)),
                     dim_ordering='tf', # default
                     input_shape=(160, 320, 3)))

model.add(Lambda(lambda x: (x / 255.0) - 0.5))

model.add(Convolution2D(16, 8, 8, activation='relu'))
model.add(Convolution2D(32, 5, 5, activation='relu'))
model.add(Convolution2D(64, 5, 5, activation='relu'))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(ELU())

model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())

model.add(Dense(50))
model.add(ELU())

model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train, batch_size=128, validation_split=0.20, shuffle=True, nb_epoch=15)

model.save('model-Comm-ai_15_augmented-leftrightcam_personal_batch-128_split20.h5')
